In [1]:
import numpy as np
import jax.numpy as jnp

import pandas as pd

import estimagic as em

rng = np.random.default_rng(seed=0)

/tmp/ipykernel_28979/1771416802.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [215]:
options = {
    # Set the number of points at which criterion is evaluated
    # in the exploration phase
    "n_samples": 10 * len(start_params_arr),
    # Pass in a DataFrame or array with a custom sample
    # for the exploration phase.
    "sample": None,
    # Determine number of optimizations, relative to n_samples
    "share_optimizations": 0.1,
    # Determine distribution from which sample is drawn
    "sampling_distribution": "uniform",
    # Determine sampling method. Allowed: ["sobol", "random",
    # "halton", "hammersley", "korobov", "latin_hypercube"]
    "sampling_method": "sobol",
    # Determine how start parameters for local optimizations are
    # calculated. Allowed: ["tiktak", "linear"] or a custom
    # function with arguments iteration, n_iterations, min_weight,
    # and max_weight
    "mixing_weight_method": "tiktak",
    # Determine bounds on mixing weights.
    "mixing_weight_bounds": (0.1, 0.995),
    # Determine after how many re-discoveries of the currently best
    # local optimum the multistart optimization converges.
    "convergence.max_discoveries": 2,
    # Determine the maximum relative distance two parameter vectors
    # can have to be considered equal for convergence purposes:
    "convergence.relative_params_tolerance": 0.01,
    # Determine how many cores are used
    "n_cores": 2,
    # Determine which batch_evaluator is used:
    "batch_evaluator": "joblib",
    # Determine the batch size. It must be larger than n_cores.
    # Setting the batch size larger than n_cores allows to reproduce
    # the exact results of a highly parallel optimization on a smaller
    # machine.
    "batch_size": 4,
    # Set the random seed:
    "seed": None,
    # Set how errors are handled during the exploration phase:
    "exploration_error_handling": "continue",
    # Set how errors are handled during the optimization phase:
    "optimization_error_handling": "continue",
}

In [209]:
# 1)
# simulate data 
# calculate moments

# -> empirical_moments

In [210]:
def simulate_data(params, n_draws, rng):
    x = rng.normal(0, 1, size=n_draws)
    e = rng.normal(0, params.loc["sd", "value"], size=n_draws)
    y = params.loc["intercept", "value"] + params.loc["slope", "value"] * x + e
    return pd.DataFrame({"y": y, "x": x})

In [211]:
true_params = pd.DataFrame(
    data=[[2, -np.inf], [-1, -np.inf], [1, 1e-10]],
    columns=["value", "lower_bound"],
    index=["intercept", "slope", "sd"],
)

data = simulate_data(true_params, n_draws=100, rng=rng)

In [212]:
def calculate_moments(sample):
    moments = {
        "y_mean": sample["y"].mean(),
        "x_mean": sample["x"].mean(),
        "yx_mean": (sample["y"] * sample["x"]).mean(),
        "y_sqrd_mean": (sample["y"] ** 2).mean(),
        "x_sqrd_mean": (sample["x"] ** 2).mean(),
    }
    return pd.Series(moments)

In [213]:
empirical_moments = calculate_moments(data)
empirical_moments

y_mean         1.711286
x_mean         0.164741
yx_mean       -0.937229
y_sqrd_mean    5.058530
x_sqrd_mean    1.069038
dtype: float64

In [7]:
# Calculate the covariance matrix of empirical moments

In [8]:
moments_cov = em.get_moments_cov(
    data, calculate_moments, bootstrap_kwargs={"n_draws": 5_000, "seed": 0}
)

moments_cov

,y_mean,x_mean,yx_mean,y_sqrd_mean,x_sqrd_mean
y_mean,0.016990,-0.008036,-0.012758,0.056742,-0.001232
x_mean,-0.008036,0.008401,0.015476,-0.029308,0.001385
yx_mean,-0.012758,0.015476,0.050458,-0.074388,-0.011105
y_sqrd_mean,0.056742,-0.029308,-0.074388,0.238695,0.009082
x_sqrd_mean,-0.001232,0.001385,-0.011105,0.009082,0.012331


In [9]:
# Define a function to calculate simulated moments

In [10]:
def simulate_moments(params, n_draws=10_000, seed=0):
    rng = np.random.default_rng(seed)
    sim_data = simulate_data(params, n_draws, rng)
    sim_moments = calculate_moments(sim_data)
    return sim_moments

In [35]:
simulate_moments(true_params)

y_mean         1.996739
x_mean         0.006312
yx_mean       -0.997919
y_sqrd_mean    5.999877
x_sqrd_mean    0.996197
dtype: float64

In [37]:
moms_data = simulate_moments(true_params)
moms_data

y_mean         1.996739
x_mean         0.006312
yx_mean       -0.997919
y_sqrd_mean    5.999877
x_sqrd_mean    0.996197
dtype: float64

In [12]:
# 5. Estimate the model parameters

In [13]:
true_params = pd.DataFrame(
    data=[[2, -np.inf], [-1, -np.inf], [1, 1e-10]],
    columns=["value", "lower_bound"],
    index=["intercept", "slope", "sd"],
)

In [14]:
true_params

,value,lower_bound
intercept,2,-inf
slope,-1,-inf
sd,1,1.000000e-10


In [15]:
true_params.assign(value=[100, 100, 100])

,value,lower_bound
intercept,100,-inf
slope,100,-inf
sd,100,1.000000e-10


In [16]:
start_params = true_params.assign(value=[100, 100, 100])

In [22]:
moments_cov

,y_mean,x_mean,yx_mean,y_sqrd_mean,x_sqrd_mean
y_mean,0.016990,-0.008036,-0.012758,0.056742,-0.001232
x_mean,-0.008036,0.008401,0.015476,-0.029308,0.001385
yx_mean,-0.012758,0.015476,0.050458,-0.074388,-0.011105
y_sqrd_mean,0.056742,-0.029308,-0.074388,0.238695,0.009082
x_sqrd_mean,-0.001232,0.001385,-0.011105,0.009082,0.012331


In [72]:
moments_cov

,y_mean,x_mean,yx_mean,y_sqrd_mean,x_sqrd_mean
y_mean,0.016990,-0.008036,-0.012758,0.056742,-0.001232
x_mean,-0.008036,0.008401,0.015476,-0.029308,0.001385
yx_mean,-0.012758,0.015476,0.050458,-0.074388,-0.011105
y_sqrd_mean,0.056742,-0.029308,-0.074388,0.238695,0.009082
x_sqrd_mean,-0.001232,0.001385,-0.011105,0.009082,0.012331


In [73]:
res = em.estimate_msm(
    simulate_moments,
    empirical_moments,
    np.eye(5),
    start_params,
    optimize_options="tranquilo",
)


In [21]:
true_params

,value,lower_bound
intercept,2,-inf
slope,-1,-inf
sd,1,1.000000e-10


In [74]:
res.summary()

,value,standard_error,ci_lower,ci_upper,p_value,free,stars
intercept,1.869538,1.005494,-0.101194,3.840270,0.062981,True,*
slope,-0.722006,1.014153,-2.709710,1.265698,0.476508,True,
sd,1.099753,1.909376,-2.642556,4.842062,0.564632,True,


In [17]:
res.summary()

,value,standard_error,ci_lower,ci_upper,p_value,free,stars
intercept,1.869534,0.130853,1.613067,2.126001,2.626213e-46,True,***
slope,-0.721958,0.231409,-1.175511,-0.268405,1.809494e-03,True,***
sd,1.099789,0.137995,0.829324,1.370254,1.589408e-15,True,***


In [ ]:
def simulate_moments(params, n_draws=10_000, seed=0):
    rng = np.random.default_rng(seed)
    sim_data = simulate_data(params, n_draws, rng)
    sim_moments = calculate_moments(sim_data)
    return sim_moments

In [30]:
rng = np.random.default_rng(seed=0)
sim_data = simulate_data(params=true_params, n_draws=10_000, rng=rng)
sim_moments = calculate_moments(sim_data)

In [34]:
jnp.array(sim_moments)

Array([ 1.9967391 ,  0.00631189, -0.99791896,  5.9998775 ,  0.9961973 ],      dtype=float32)

In [42]:
W_hat = np.eye(5)

In [113]:
def simulate_simple(params, n_draws=10_000, seed=0):
    
    rng = np.random.default_rng(seed)
    sim_data = simulate_data(params, n_draws, rng)
    sim_moments = calculate_moments(sim_data)
    moms_model = np.array(sim_moments)
    
    #moms_data = 
    
    err = moms_model - np.array(moms_data)
    crit_val = np.dot(np.dot(err.T, W_hat), err)
    
    chol_weights = W_hat
    deviations = moms_model - np.array(moms_data)
    root_contribs = deviations @ chol_weights
    
    # return root_contribs
    return {"root_contributions": root_contribs, "value": crit_val}

In [114]:
crit = simulate_simple(start_params)
crit

{'root_contributions': array([   98.93954859,     0.        ,    99.81428543, 29736.64679207,
            0.        ]),
 'value': 884287914.3622243}

In [157]:
start_params_zero = start_params.assign(value=[10, 10, 100])

In [181]:
start_params_zero_bound = start_params_zero.assign(lower_bound=[-100, -100, 0.1])
start_params_zero_bound["upper_bound"] = [100, 100, 1_000]

In [182]:
start_params_zero_bound

,value,lower_bound,upper_bound
intercept,10,-100.0,100
slope,10,-100.0,100
sd,100,0.1,1000


In [203]:
algo_options = {
    "convergence.relative_criterion_tolerance": 1e-12,
    "stopping.max_iterations": 100,
}

In [217]:
out = em.minimize(
    criterion=simulate_simple,
    params=start_params_zero_bound,
    #algorithm="scipy_lbfgsb",
    algorithm="tranquilo_ls",
    algo_options=algo_options,
    multistart=True,
    multistart_options=options
    
)

In [218]:
out.params

,value,lower_bound,upper_bound
intercept,2.0,-100.0,100
slope,-1.0,-100.0,100
sd,1.0,0.1,1000


In [190]:
out

/home/sebastian/miniconda3/envs/elder_care/lib/python3.10/site-packages/estimagic/optimization/optimize_result.py:131: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  table = report[columns].applymap(_format_float).astype(str)


Minimize with 3 free parameters terminated.

The value of criterion improved from 101055319.8559496 to 5.342975928336405e-19.

The multistart_tranquilo_ls algorithm reported: Relative criterion change smaller than tolerance.

Independent of the convergence criteria used by multistart_tranquilo_ls, the strength of convergence can be assessed by the following criteria:

                             one_step     five_steps 
relative_criterion_change  9.428e-15***  9.428e-15***
relative_params_change     1.592e-08*    1.592e-08*  
absolute_criterion_change  9.428e-16***  9.428e-16***
absolute_params_change     1.592e-08*    1.592e-08*  

(***: change <= 1e-10, **: change <= 1e-8, *: change <= 1e-5. Change refers to a change between accepted steps. The first column only considers the last step. The second column considers the last five steps.)

In [230]:
res = em.estimate_msm(
    simulate_moments,
    empirical_moments,
    np.eye(5),
    params=start_params_zero_bound,
    optimize_options={
        "algorithm": "tranquilo_ls",
        "algo_options": algo_options,
        "multistart": True,
        "multistart_options": options
    },
)

In [231]:
res.params

,value,lower_bound,upper_bound
intercept,1.713790,-100.0,100
slope,-0.935611,-100.0,100
sd,1.114875,0.1,1000


In [156]:
true_params

,value,lower_bound
intercept,2,-inf
slope,-1,-inf
sd,1,1.000000e-10


In [90]:
start_params_arr = np.array(start_params)[:, 0]

In [101]:
start_params_bound = start_params.assign(lower_bound=[-100, -100, -100])
start_params_bound["upper_bound"] = [1_000, 1_000, 1_000]

In [102]:
start_params_bound

,value,lower_bound,upper_bound
intercept,100,-100,1000
slope,100,-100,1000
sd,100,-100,1000


#### def simulate_moments(params, n_draws=10_000, seed=0)

endog, value, policy_left, policy_right = solve_partial()

dict = simulate()

df = custom_create_simulation_df()

moments = custom_create_simulated_moments(df) # or turned into jnp.array before